In [1]:
import pandas as pd
import re
import string
import nltk
import math
from nltk.tokenize import sent_tokenize
from textblob import TextBlob
from nltk.stem.snowball import SnowballStemmer
from spellchecker import SpellChecker
from textblob import Word

In [2]:
df = pd.read_csv("result.csv")

In [3]:
df

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,JobTitle,Company,Location,PostDate,ExtractDate,Summary,JobUrl
0,0,0.0,0.0,0.0,.Net Developer,SmartDev,Ðà Nẵng,Posted\nPosted 12 days ago,2023-01-22,You will report to PM (Project Manager).\nColl...,.NET Developer
1,1,1.0,1.0,1.0,HCMC - .NET/Reactjs Developer (All levels),Amaris Consulting,Thành phố Hồ Chí Minh,Posted\nPosted 2 days ago,2023-01-22,"Case study: Depending on the position, we may ...",.NET Developer
2,2,2.0,2.0,2.0,"Web Developer (.NET, ASP.NET, C#)",NPV Corp,Ðà Nẵng,Posted\nPosted 13 days ago,2023-01-22,Chi tiết trao đổi trong quá trình phỏng vấn.\n...,.NET Developer
3,3,3.0,3.0,3.0,"Software Developer (C#, .NET, SQL)",Keystone Solutions,Ðà Nẵng,Posted\nPosted 21 days ago,2023-01-22,Thực hiện Unit Test dựa theo tải liệu Unit Tes...,.NET Developer
4,4,4.0,4.0,4.0,IT Developer,Accredo Asia Packaging,Thủ Dầu Một,Posted\nPosted 30+ days ago,2023-01-22,"Xây dựng mới phần mềm và ứng dụng.\nNet, Windo...",.NET Developer
...,...,...,...,...,...,...,...,...,...,...,...
2549,147,NaN,NaN,NaN,EJV_Embedded Software Specialist (Senior P-Res...,Bosch Group,Thành phố Hồ Chí Minh,Posted\nPosted 17 days ago,2023-01-24,Design and implement software of embedded devi...,Embedded Engineer\n
2550,148,NaN,NaN,NaN,[EDA8] Embedded Software Test Engineer,Bosch Group,Thành phố Hồ Chí Minh,Posted\nPosted 28 days ago,2023-01-24,Basic knowledge of Micro-controller and embedd...,Embedded Engineer\n
2551,149,NaN,NaN,NaN,CI/XDV_Fresher Software Test Engineer (Manual ...,Bosch Group,Thành phố Hồ Chí Minh,Posted\nPosted 18 days ago,2023-01-24,This engineering development center will be en...,Embedded Engineer\n
2552,150,NaN,NaN,NaN,Senior/Regular Embedded C++ Developer,Luxoft,Thành phố Hồ Chí Minh,Posted\nPosted 30+ days ago,2023-01-24,Analysis and Design experience for embedded SW...,Embedded Engineer\n


In [77]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [78]:
len(df['title'].unique())

906

In [79]:
len(df.loc[df.duplicated() == False, :])

958

In [80]:
df['title'].unique()

array(['Service Delivery Manager – Well Intervention (Vietnam)',
       'Associate Risk Assurance (IT Risk)',
       'Database Administrator (SQL, Oracle)', 'Senior HRIS Executive',
       'Key Account Manager', '[EDA6] Test Coordinator/ Test Lead',
       'Deputy Manager - Sales, Polyamide and Precursors, Monomers',
       'Senior Counsel and Data Privacy Officer',
       'Frontend Software Engineer', 'DevOps (Blockchain Gaming)',
       'Azure Cloud Admin', 'Cloud Engineer',
       'Senior IT Technical Support Engineer',
       'Remote SR. Fullstack Developer .Net, C#, Nodejs',
       'Automation Project Engineer', '[EMC] Technical Program Manager',
       'Data Analysis', '.Net Developer',
       'Java Developer – Senior/ Middle', 'IT Risk and BCM Manager',
       'BLOCKCHAIN ENGINEER', 'Embedded Developer C/C++',
       'Business Development Manager',
       'Engineering Manager, KMS Healthcare',
       'Java Software Engineer (Vietnam & India), KMS Solutions',
       'Manager EY P

In [81]:
df['title'] = df['title'].str.lower()

In [82]:
df['title'].unique()

array(['service delivery manager – well intervention (vietnam)',
       'associate risk assurance (it risk)',
       'database administrator (sql, oracle)', 'senior hris executive',
       'key account manager', '[eda6] test coordinator/ test lead',
       'deputy manager - sales, polyamide and precursors, monomers',
       'senior counsel and data privacy officer',
       'frontend software engineer', 'devops (blockchain gaming)',
       'azure cloud admin', 'cloud engineer',
       'senior it technical support engineer',
       'remote sr. fullstack developer .net, c#, nodejs',
       'automation project engineer', '[emc] technical program manager',
       'data analysis', '.net developer',
       'java developer – senior/ middle', 'it risk and bcm manager',
       'blockchain engineer', 'embedded developer c/c++',
       'business development manager',
       'engineering manager, kms healthcare',
       'java software engineer (vietnam & india), kms solutions',
       'manager ey p

In [83]:
df['titleSave'] = df['title']

In [84]:
# xóa hết tất cả có kí tự số trong cột title
df['title'] = df['title'].str.replace('\d+',' ')

C:\Users\Tran Huu An\AppData\Local\Temp\ipykernel_6648\3736478334.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['title'] = df['title'].str.replace('\d+',' ')


In [85]:
def replaceString(string):
    if (string.rfind('(') >=0) & (string.rfind(')') >=0):
        indexStr1 = string.index('(')
        indexStr2 = string.index(')')
        char = string[indexStr1: indexStr2+1]
        string = string.replace(char, '')
    if (string.rfind('[') >= 0) & (string.rfind(']') >= 0):
        indexStr3 = string.index('[')
        indexStr4 = string.index(']')
        char1 = string[indexStr3: indexStr4+1]
        string = string.replace(char1, '')
    return string

In [86]:
replaceString('(senior) security product owner')

' security product owner'

In [87]:
df['title'] = df['title'].apply(lambda x: replaceString(x))

In [88]:
df['title'].unique()

array(['service delivery manager – well intervention ',
       'associate risk assurance ', 'database administrator ',
       'senior hris executive', 'key account manager',
       ' test coordinator/ test lead',
       'deputy manager - sales, polyamide and precursors, monomers',
       'senior counsel and data privacy officer',
       'frontend software engineer', 'devops ', 'azure cloud admin',
       'cloud engineer', 'senior it technical support engineer',
       'remote sr. fullstack developer .net, c#, nodejs',
       'automation project engineer', ' technical program manager',
       'data analysis', '.net developer',
       'java developer – senior/ middle', 'it risk and bcm manager',
       'blockchain engineer', 'embedded developer c/c++',
       'business development manager',
       'engineering manager, kms healthcare',
       'java software engineer , kms solutions',
       'manager ey parthenon - hanoi office', 'business analyst',
       'sales engineer, life science/',

In [89]:
# def removeSpecialString(special_string):
#     # Calling translate() method
#     normal_string = special_string.translate(str.maketrans('', '', string.punctuation))
#     return normal_string

In [90]:
def removeSpecialString(string):
    string = re.sub('\W+',' ', string )
    return string

In [91]:
df['title'] = df['title'].apply(lambda x: removeSpecialString(x))

In [92]:
df['title'].unique()

array(['service delivery manager well intervention ',
       'associate risk assurance ', 'database administrator ',
       'senior hris executive', 'key account manager',
       ' test coordinator test lead',
       'deputy manager sales polyamide and precursors monomers',
       'senior counsel and data privacy officer',
       'frontend software engineer', 'devops ', 'azure cloud admin',
       'cloud engineer', 'senior it technical support engineer',
       'remote sr fullstack developer net c nodejs',
       'automation project engineer', ' technical program manager',
       'data analysis', ' net developer', 'java developer senior middle',
       'it risk and bcm manager', 'blockchain engineer',
       'embedded developer c c ', 'business development manager',
       'engineering manager kms healthcare',
       'java software engineer kms solutions',
       'manager ey parthenon hanoi office', 'business analyst',
       'sales engineer life science ',
       'vn_bgsv_supply chain

In [93]:
df['title'] = df['title'].str.strip()

In [94]:
df['title'].unique()

array(['service delivery manager well intervention',
       'associate risk assurance', 'database administrator',
       'senior hris executive', 'key account manager',
       'test coordinator test lead',
       'deputy manager sales polyamide and precursors monomers',
       'senior counsel and data privacy officer',
       'frontend software engineer', 'devops', 'azure cloud admin',
       'cloud engineer', 'senior it technical support engineer',
       'remote sr fullstack developer net c nodejs',
       'automation project engineer', 'technical program manager',
       'data analysis', 'net developer', 'java developer senior middle',
       'it risk and bcm manager', 'blockchain engineer',
       'embedded developer c c', 'business development manager',
       'engineering manager kms healthcare',
       'java software engineer kms solutions',
       'manager ey parthenon hanoi office', 'business analyst',
       'sales engineer life science',
       'vn_bgsv_supply chain logistic

In [95]:
# sử dụng TF-IDF để lọc những từ không quan trọng
def tf(tem, doc):
    result  = 0
    doc = doc.split(' ')
    for word in doc:
        if word == tem:
            result +=1
    result  = result / len(doc)
    return round(result, 5)

In [96]:
tf('chain', 'vn_bgsv_supply chain logistics import export responsible chain chain chain chain')

0.5

In [97]:
def idf(term, docs):
    result = 0
    for doc in docs:
        doc = doc.split(" ")
        for word in doc:
            if word == term:
                result += 1
                break
    return math.log(len(docs) / result, math.e)

In [98]:
resultIdf = idf('chain', df['title'].unique())

In [99]:
def tf_idf(term, doc, docs):
    return tf(term, doc) * idf(term, docs)

In [100]:
tf_idf('chain', 'vn_bgsv_supply chain logistics import export responsible', df['title'].unique())

0.8534147609630894

In [101]:
spell = SpellChecker()

missedSpell = spell.unknown(df['title'])


In [102]:
def missSpellSting(textvaul):
    
    new_doc = TextBlob(textvaul)

    result = new_doc.correct()

    return str(result)

In [103]:
missSpellSting('senior c software engineer salary up to')

'senior c software engineer salary up to'

In [104]:
df['title1'] = df['title'].apply(lambda text: missSpellSting(text))

In [105]:
df['title1']

0           service delivery manager well intervention
1                             associate risk assurance
2                               database administrator
3                                 senior his executive
4                                  key account manager
                             ...                      
1001    country business director power tools division
1002                                 flutter developer
1003                     was cloud solutions architect
1004                                 software engineer
1005          cloud inside sales representative vienna
Name: title1, Length: 1006, dtype: object

In [106]:
spell.unknown(df['title1'])

{'a engineer',
 'a engineers',
 'a inter',
 'a specialist',
 'a tested',
 'account assistant',
 'account manager',
 'accountant view him mi',
 'accounting and office admit',
 'accounting associate',
 'admit and secretary inter eye',
 'admit executive and receptions',
 'admit inter',
 'admit inter for department',
 'adviser child safety',
 'ai expert',
 'ai researcher',
 'analogy design engineer',
 'analyst it iii nm',
 'andros developer',
 'andros engineer',
 'andros software engineer',
 'apace controller and enterprise',
 'application consulting inter',
 'application support engineer',
 'application supporter',
 'arm assistant internship him',
 'arm senior data analyst',
 'assistant e commerce manager',
 'assistant manager it core system',
 'associate art producer',
 'associate creative director',
 'associate information technology',
 'associate it risk assurance',
 'associate risk assurance',
 'associate saber security',
 'automatic project engineer',
 'automatic tested',
 'automatic

In [107]:
df['title1'] = df['title1'].apply(lambda text: missSpellSting(text))

In [108]:
len(df['title1'].unique())

831

In [109]:
print(df['title1'].unique())

['service delivery manager well intervention' 'associate risk assurance'
 'database administrator' 'senior his executive' 'key account manager'
 'test coordinate test lead'
 'deputy manager sales polyamide and precursor monsters'
 'senior counsel and data privacy officer' 'fronted software engineer'
 'drops' 'azure cloud admit' 'cloud engineer'
 'senior it technical support engineer'
 'remote s fullstack developer net c nodes' 'automatic project engineer'
 'technical program manager' 'data analysis' 'net developer'
 'cava developer senior middle' 'it risk and cm manager'
 'blockchain engineer' 'embedded developer c c'
 'business development manager' 'engineering manager ms healthcare'
 'cava software engineer ms solutions' 'manager by parthenon hand office'
 'business analyst' 'sales engineer life science'
 'vn_bgsv_supply chain logistics import export responsible'
 'software development manager' 'when in it'
 'senior medium cava engineer messagemedia' 'data architect'
 'd model artist

In [110]:
gfg = Word("important")

In [111]:
# using Word.spellcheck() method
print(gfg.spellcheck())

[('important', 1.0)]


In [112]:
def missValue(string):
    textVaul = ''
    string = string.split(" ")
    for word in string:
        gfg = Word(word)
        result = gfg.spellcheck()[0][0]
        textVaul += result
        textVaul += " "
    textVaul = textVaul.strip()
    return textVaul

In [113]:
missValue('vn_bgsv_supply chain logistics import export responsible')

'vn_bgsv_supply chain logistics import export responsible'

In [114]:
def missSpellSting(textvaul):
    
    new_doc = TextBlob(textvaul)

    correctResult = new_doc.correct()
    result  = missValue(correctResult)
    return str(result)

In [115]:
df['title1'] = df['title1'].apply(lambda text: missSpellSting(text))

In [116]:
def get_tf_idf(string, docs):
    duplicates = []
    dictionary = {}
    result = []
    words = string.split()
    for i in words:
#         if i not in duplicates:
#         duplicates.append(i)
        core = tf_idf(i, string, docs)
#        dictionary[i] = core
        result.append(core)
    return result

In [117]:
get_tf_idf('him middle senior cava developer him him c', df['title1'])

[1.3299155702524539,
 0.6909303736924742,
 0.2626941244109084,
 0.4349687682718173,
 0.25577536252551275,
 1.3299155702524539,
 1.3299155702524539,
 0.45223756058191944]

In [118]:
df['titleCore'] = df['title1'].apply(lambda string: get_tf_idf(string, df['title1']))

In [119]:
df[['titleCore', 'title1']].head(1)

,titleCore,title1
0,"[0.7738429825872524, 0.9935654403208742, 0.439...",service delivery manager well intervention


In [120]:
df['titleCore'][9]

[5.304299438225584]

In [121]:
df['title1'][9]

'drops'

In [122]:
x = {}

In [123]:
x['an'] = 1
x['bb'] = 0.56

In [124]:
for x, y in x.items():
    print(y)

1
0.56


In [125]:
def get_tf_idf1(string, docs):
    duplicates = []
    dictionary = {}
    fixText = ''
    words = string.split()
    for i in words:
#         if i not in duplicates:
#         duplicates.append(i)
        core = tf_idf(i, string, docs)
        dictionary[i] = core
    for key, val in dictionary.items():
        if val > 4:
            fixText += key
            fixText += ' '
    return fixText

In [126]:
df['title'][5]

'test coordinator test lead'

In [127]:
get_tf_idf1('test coordinator test lead', df['title'])

''

In [128]:
def get_tf_idf1(string, docs):
    duplicates = []
    dictionary = {}
    result = []
    fixText = ''
    words = string.split()
    for i in words:
        core = tf_idf(i, string, docs)
        result.append(core)
        dictionary[i] = core
        print(result)
    for key, val in dictionary.items():
        if val > 0.7:
            fixText += key
            fixText += ' '
    fixText = fixText.strip()
    return fixText

In [129]:
df['titleFix'] = df['title1'].apply(lambda string: get_tf_idf1(string, df['title1']))

[0.7738429825872524]
[0.7738429825872524, 0.9935654403208742]
[0.7738429825872524, 0.9935654403208742, 0.43904769587291803]
[0.7738429825872524, 0.9935654403208742, 0.43904769587291803, 1.244118034019948]
[0.7738429825872524, 0.9935654403208742, 0.43904769587291803, 1.244118034019948, 1.244118034019948]
[1.1708349478659466]
[1.1708349478659466, 1.3059886323516408]
[1.1708349478659466, 1.3059886323516408, 1.4762621375215585]
[2.763721494769897]
[2.763721494769897, 2.5609889407158146]
[0.7005106599191048]
[0.7005106599191048, 1.9383556369136516]
[0.7005106599191048, 1.9383556369136516, 1.0920394764737964]
[1.9383556369136516]
[1.9383556369136516, 1.4248790914147385]
[1.9383556369136516, 1.4248790914147385, 0.7317388423265988]
[2.4171479044899242]
[2.4171479044899242, 1.1791281933308662]
[2.4171479044899242, 1.1791281933308662, 2.4171479044899242]
[2.4171479044899242, 1.1791281933308662, 2.4171479044899242, 0.9795012692764233]
[0.8307487663561163]
[0.8307487663561163, 0.3136117691620253]


[0.7005106599191048, 0.898067567729041]
[0.7005106599191048, 0.898067567729041, 0.9288583665283123]
[1.7284343376649212]
[1.7284343376649212, 0.8866103801683026]
[1.7284343376649212, 0.8866103801683026, 1.5551475425249348]
[1.7284343376649212, 0.8866103801683026, 1.5551475425249348, 0.5488096198411475]
[0.9031684155722628]
[0.9031684155722628, 1.244118034019948]
[0.9031684155722628, 1.244118034019948, 1.382747470131937]
[0.9031684155722628, 1.244118034019948, 1.382747470131937, 0.8046731185527041]
[0.9031684155722628, 1.244118034019948, 1.382747470131937, 0.8046731185527041, 0.9433025546646929]
[1.3471148227417888]
[1.3471148227417888, 1.3933014828072965]
[2.0735093213993463]
[2.0735093213993463, 1.4248790914147385]
[2.0735093213993463, 1.4248790914147385, 0.7317388423265988]
[2.01168279638176]
[2.01168279638176, 2.5609889407158146]
[0.7005106599191048]
[0.7005106599191048, 1.5370353820476874]
[0.7005106599191048, 1.5370353820476874, 0.6005771816025061]
[0.7005106599191048]
[0.70051065

[0.9795012692764233, 0.5253882488218168, 0.6663105271525814]
[0.9795012692764233, 0.5253882488218168, 0.6663105271525814, 0.4504373905757853]
[1.1821353520860014]
[1.1821353520860014, 2.0735093213993463]
[1.1821353520860014, 2.0735093213993463, 1.9383556369136516]
[0.5253882488218168]
[0.5253882488218168, 1.0871969982995369]
[0.5253882488218168, 1.0871969982995369, 1.5551475425249348]
[0.5253882488218168, 1.0871969982995369, 1.5551475425249348, 1.5551475425249348]
[2.652149719112792]
[2.652149719112792, 0.9008747811515706]
[1.611415822007253]
[1.611415822007253, 0.7317388423265988]
[1.611415822007253, 0.7317388423265988, 2.0735093213993463]
[1.3471148227417888]
[1.3471148227417888, 1.3933014828072965]
[0.9445607836826337]
[0.9445607836826337, 1.2804944703579073]
[0.9445607836826337, 1.2804944703579073, 0.5253882488218168]
[0.9445607836826337, 1.2804944703579073, 0.5253882488218168, 0.4504373905757853]
[1.0367857636505236]
[1.0367857636505236, 0.6158225705036666]
[1.0367857636505236, 0.

[0.31491102449840574, 0.4433051900841513, 0.8642171688324606, 0.8642171688324606, 0.8642171688324606, 0.7775737712624674, 0.6909303736924742, 0.5763940322082048]
[1.4762621375215585]
[1.4762621375215585, 2.0735093213993463]
[1.4762621375215585, 2.0735093213993463, 0.9627855920226455]
[0.35026583772452885]
[0.35026583772452885, 0.7526553991171453]
[0.35026583772452885, 0.7526553991171453, 0.3658803973556963]
[0.35026583772452885, 0.7526553991171453, 0.3658803973556963, 0.7124609191074746]
[0.35026583772452885, 0.7526553991171453, 0.3658803973556963, 0.7124609191074746, 0.7381532129142836]
[0.35026583772452885, 0.7526553991171453, 0.3658803973556963, 0.7124609191074746, 0.7381532129142836, 0.6029954737775082]
[0.5253882488218168]
[0.5253882488218168, 1.4537812654978937]
[0.5253882488218168, 1.4537812654978937, 1.2804944703579073]
[0.5253882488218168, 1.4537812654978937, 1.2804944703579073, 0.8190377977333246]
[1.1821353520860014]
[1.1821353520860014, 1.768082131743734]
[1.182135352086001

[1.3803690723112063, 1.2897254069290442, 1.4495815017727385]
[0.7005106599191048]
[0.7005106599191048, 0.898067567729041]
[0.7005106599191048, 0.898067567729041, 0.9288583665283123]
[1.4471601938621423]
[1.4471601938621423, 1.4471601938621423]
[1.4471601938621423, 1.4471601938621423, 0.755648626946107]
[1.4471601938621423, 1.4471601938621423, 0.755648626946107, 0.5330484217220651]
[1.4471601938621423, 1.4471601938621423, 0.755648626946107, 0.5330484217220651, 0.8697575986396295]
[1.0367857636505236]
[1.0367857636505236, 0.7009618772167443]
[1.0367857636505236, 0.7009618772167443, 0.34104063737701773]
[1.0367857636505236, 0.7009618772167443, 0.34104063737701773, 0.6029954737775082]
[1.0367857636505236, 0.7009618772167443, 0.34104063737701773, 0.6029954737775082, 0.5910854082506041]
[1.0367857636505236, 0.7009618772167443, 0.34104063737701773, 0.6029954737775082, 0.5910854082506041, 0.8536800534982097]
[0.6500351409060077]
[0.6500351409060077, 0.9222304515331278]
[0.6500351409060077, 0.9

...

[0.7712739885426474, 0.8257048595038651, 1.0686700052611064]
[0.7712739885426474, 0.8257048595038651, 1.0686700052611064, 1.7284343376649212]
[1.5425479770852948]
[1.5425479770852948, 1.7398750730872692]
[1.259644097993623]
[1.259644097993623, 2.01168279638176]
[0.8866103801683026]
[0.8866103801683026, 0.7712739885426474]
[0.8866103801683026, 0.7712739885426474, 0.8257048595038651]
[0.8866103801683026, 0.7712739885426474, 0.8257048595038651, 0.4504373905757853]
[0.8397543343684287]
[0.8397543343684287, 1.1009288032736935]
[0.8397543343684287, 1.1009288032736935, 0.6005771816025061]
[1.326074859556396]
[1.326074859556396, 0.7220964149811339]
[1.326074859556396, 0.7220964149811339, 1.107207675217921]
[1.326074859556396, 0.7220964149811339, 1.107207675217921, 1.326074859556396]
[1.0833810675909976]
[1.0833810675909976, 1.4495815017727385]
[1.0833810675909976, 1.4495815017727385, 1.768082131743734]
[1.2316081923920752]
[1.2316081923920752, 1.2059547685501697]
[1.2316081923920752, 1.2059547

[1.1821353520860014]
[1.1821353520860014, 2.0735093213993463]
[1.1821353520860014, 2.0735093213993463, 1.3601610671211342]
[1.4537812654978937]
[1.4537812654978937, 1.326074859556396]
[1.4537812654978937, 1.326074859556396, 1.7284343376649212]
[1.4537812654978937, 1.326074859556396, 1.7284343376649212, 1.5551475425249348]
[0.7005106599191048]
[0.7005106599191048, 2.304556071095393]
[0.7005106599191048, 2.304556071095393, 0.6005771816025061]
[1.0352871571049758]
[1.0352871571049758, 0.9673037282340654]
[1.0352871571049758, 0.9673037282340654, 1.0871969982995369]
[1.0352871571049758, 0.9673037282340654, 1.0871969982995369, 1.00584139819088]
[0.5253882488218168]
[0.5253882488218168, 1.7284343376649212]
[0.5253882488218168, 1.7284343376649212, 0.7712739885426474]
[0.5253882488218168, 1.7284343376649212, 0.7712739885426474, 1.7284343376649212]
[0.8857661401743369]
[0.8857661401743369, 0.5330484217220651]
[0.8857661401743369, 0.5330484217220651, 0.8697575986396295]
[0.8857661401743369, 0.533

[0.5916044930560673, 0.5168532632378641]
[0.5916044930560673, 0.5168532632378641, 0.9876965179152425]
[0.5916044930560673, 0.5168532632378641, 0.9876965179152425, 0.9514767025506833]
[0.5916044930560673, 0.5168532632378641, 0.9876965179152425, 0.9514767025506833, 0.5397598142276042]
[0.5916044930560673, 0.5168532632378641, 0.9876965179152425, 0.9514767025506833, 0.5397598142276042, 0.9514767025506833]
[0.5916044930560673, 0.5168532632378641, 0.9876965179152425, 0.9514767025506833, 0.5397598142276042, 0.9514767025506833, 0.533679747711735]
[0.6008244661857807]
[0.6008244661857807, 0.3807564876360711]
[0.6008244661857807, 0.3807564876360711, 0.25739794247062675]
[0.6008244661857807, 0.3807564876360711, 0.25739794247062675, 0.9876965179152425]
[0.6008244661857807, 0.3807564876360711, 0.25739794247062675, 0.9876965179152425, 0.6451331992432673]
[0.6008244661857807, 0.3807564876360711, 0.25739794247062675, 0.9876965179152425, 0.6451331992432673, 0.7577722177449069]
[0.6008244661857807, 0.38

[0.7685407467851321, 0.9212589230665975]
[0.7685407467851321, 0.9212589230665975, 0.8057320824826714]
[0.7685407467851321, 0.9212589230665975, 0.8057320824826714, 0.7381532129142836]
[0.7685407467851321, 0.9212589230665975, 0.8057320824826714, 0.7381532129142836, 0.34104063737701773]
[0.7685407467851321, 0.9212589230665975, 0.8057320824826714, 0.7381532129142836, 0.34104063737701773, 0.48140723793962237]
[1.4812468160391286]
[1.4812468160391286, 1.3933014828072965]
[1.4248790914147385]
[1.4248790914147385, 1.707308887217605]
[1.4248790914147385, 1.707308887217605, 0.6005771816025061]
[1.4537812654978937]
[1.4537812654978937, 1.3818607473849485]
[1.4537812654978937, 1.3818607473849485, 0.8325546030508936]
[1.4537812654978937, 1.3818607473849485, 0.8325546030508936, 1.2419568004010928]
[2.0705743142099515]
[2.0705743142099515, 2.0402620033017342]
[0.9692068940821358]
[0.9692068940821358, 1.1523126042344498]
[0.9692068940821358, 1.1523126042344498, 1.0367857636505236]
[0.9692068940821358,

[0.6735574113708944, 1.3818607473849485, 0.6966507414036482]
[0.6735574113708944, 1.3818607473849485, 0.6966507414036482, 0.8190377977333246]
[1.0686700052611064]
[1.0686700052611064, 1.0871969982995369]
[1.0686700052611064, 1.0871969982995369, 0.8866103801683026]
[1.0686700052611064, 1.0871969982995369, 0.8866103801683026, 0.5115507250510255]
[2.5609889407158146]
[2.5609889407158146, 2.1743939965990737]
[0.755648626946107]
[0.755648626946107, 1.0660968434441302]
[0.755648626946107, 1.0660968434441302, 0.4092405800408204]
[0.755648626946107, 1.0660968434441302, 0.4092405800408204, 0.8857661401743369]
[0.755648626946107, 1.0660968434441302, 0.4092405800408204, 0.8857661401743369, 1.0660968434441302]
[0.8866103801683026]
[0.8866103801683026, 0.9556737243253421]
[0.8866103801683026, 0.9556737243253421, 1.107207675217921]
[0.8866103801683026, 0.9556737243253421, 1.107207675217921, 0.8125439261325095]
[1.4812468160391286]
[1.4812468160391286, 1.3933014828072965]
[1.768082131743734]
[1.76808

In [130]:
len(df['titleFix'].unique())

807

In [131]:
df['title1']

0           service delivery manager well intervention
1                             associate risk assurance
2                               database administrator
3                                 senior his executive
4                                  key account manager
                             ...                      
1001    country business director power tools division
1002                                 flutter developer
1003                     was cloud solutions architect
1004                                 software engineer
1005          cloud inside sales representative vienna
Name: title1, Length: 1006, dtype: object

In [132]:
df['titleFix'][1001]

'country director power tools division'

# handle description

In [133]:
df['description']

0       Service Delivery Manager – Well Intervention (...
1       Line of Service\nAssurance\nIndustry/Sector\nN...
2       Database Administrator (SQL, Oracle)\nTimo\nỨn...
3       Ninja Van is a tech-enabled logistics company ...
4       Design solutions to drive safe living and qual...
                              ...                        
1001    Company Description\n\nThe Bosch Group is a le...
1002    Job Description\nParticipate in analysis, desi...
1003    Khối / Ban\nKhối Dịch vụ Công nghệ Ngân hàng\n...
1004    Responsibilities and Duties\n\nDevelopment on ...
1005    Acronis is a world leader in cyber protection—...
Name: description, Length: 1006, dtype: object

In [134]:
df['description'] = df['description'].str.replace('\n', ' ')

In [135]:
df['description']

0       Service Delivery Manager – Well Intervention (...
1       Line of Service Assurance Industry/Sector Not ...
2       Database Administrator (SQL, Oracle) Timo Ứng ...
3       Ninja Van is a tech-enabled logistics company ...
4       Design solutions to drive safe living and qual...
                              ...                        
1001    Company Description  The Bosch Group is a lead...
1002    Job Description Participate in analysis, desig...
1003    Khối / Ban Khối Dịch vụ Công nghệ Ngân hàng  L...
1004    Responsibilities and Duties  Development on ap...
1005    Acronis is a world leader in cyber protection—...
Name: description, Length: 1006, dtype: object

In [144]:
df['description'] = df['description'].str.replace('\n', '. ')

In [137]:
def subString(string):
    result = re.sub('\W', ' ',string)
    result = result.replace('  ', ' ')
    return result

In [138]:
df['description'] = df['description'].apply(lambda x: subString(x))

In [146]:
stringsss = df['description'][0]

In [147]:
lst = stringsss.split('. ')

In [149]:
for i in lst:
    if i == '':
        lst.remove(i)

In [158]:
for j in range(len(lst)):
    lst[j] = subString(lst[j])

In [159]:
lst

['Service Delivery Manager Well Intervention Vietnam ',
 'Do you enjoy being part of a successful completion and well intervention team ',
 'Do you enjoy managing a team that provides a high quality of service to customers ',
 'Join our Completions Wellbore Intervention Team',
 'Our Completions and Intervention business drawing from the industry s broadest portfolio of proven products and innovative technologies provides the market leading products and service solution needed to produce safely and efficiently for the life of assets',
 'We create improvements and efficiency through service delivery remote operations automation and advanced analytics ',
 'Partner with the best',
 'As a Service Delivery Manager Well Intervention Vietnam you ll manage the service delivery for the well intervention portfolio by enabling flawless execution on time delivery contract adherence for assigned area of responsibility in a customer facing capacity ',
 'As a Service Delivery Manager Well Intervention